원본 + 좌우상하 반전 ->벡터 추출 / test는 원본으로만

In [1]:
import numpy as np
import pandas as pd
import cv2 
from PIL import Image
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

In [2]:
y = np.load('new_y.npy')
x = np.load('new_cut_feature.npy')
x_ag = np.load('new_cut_transpose_feature.npy')
x.shape , y.shape , x_ag.shape

((1900, 138), (1900,), (3800, 138))

In [3]:
x_ar = []
x_en = []

y_ar = []
y_en = []
for i,v in enumerate(x):
    if i%4<2:
        x_ar.append(v)
        y_ar.append(y[i])
    else:
        x_en.append(v)
        y_en.append(y[i])
        
x_ar , y_ar , x_en , y_en= np.array(x_ar),np.array(y_ar),np.array(x_en),np.array(y_en)

print(x_ar.shape , x_en.shape , y_ar.shape , y_en.shape)

(950, 138) (950, 138) (950,) (950,)


In [4]:
#train test = 282 : 193

#아랍 데이터 나누기
x_train_ar , x_test_ar = x_ar[:564] , x_ar[564:]
y_train_ar , y_test_ar = y_ar[:564] , y_ar[564:]


#영어 데이터 나누기
x_train_en , x_test_en = x_en[:564] , x_en[564:]
y_train_en , y_test_en = y_en[:564] , y_en[564:]


#영아 + 아랍 데이터
x_train_all = np.concatenate((x_train_ar,x_train_en),axis=0)
x_test_all =  np.concatenate((x_test_ar,x_test_en),axis=0)

y_train_all = np.concatenate((y_train_ar,y_train_en),axis=0)
y_test_all = np.concatenate((y_test_ar,y_test_en),axis=0)

print(x_train_ar.shape , x_test_ar.shape)
print(y_train_ar.shape , y_test_ar.shape)

print(x_train_en.shape , x_test_en.shape)
print(y_train_en.shape , y_test_en.shape)
      
print(x_train_all.shape , x_test_all.shape)
print(y_train_all.shape , y_test_all.shape)

(564, 138) (386, 138)
(564,) (386,)
(564, 138) (386, 138)
(564,) (386,)
(1128, 138) (772, 138)
(1128,) (772,)


In [5]:
#데이터 증강 나누기
x_ag_ar , x_ag_en = [] , []
y_ag_ar , y_ag_en = [] , []

for i , v in enumerate(x_ag[:2256]):
    idx = i//2 #몇번째 사람인지
    if i%8 < 4:
        x_ag_ar.append(v)
        y_ag_ar.append(y[idx])
    else:
        x_ag_en.append(v)
        y_ag_en.append(y[idx])
        
x_ag_ar , x_ag_en = np.array(x_ag_ar) , np.array(x_ag_en)
y_ag_ar , y_ag_en = np.array(y_ag_ar) , np.array(y_ag_en)
print(x_ag_ar.shape , x_ag_en.shape)
print(y_ag_ar.shape , y_ag_en.shape)

(1128, 138) (1128, 138)
(1128,) (1128,)


In [6]:
#원본 + 증강 합치기
x_train_ar = np.concatenate((x_train_ar,x_ag_ar),axis=0)
y_train_ar = np.concatenate((y_train_ar,y_ag_ar),axis=0)

x_train_en = np.concatenate((x_train_en,x_ag_en),axis=0)
y_train_en = np.concatenate((y_train_en,y_ag_en),axis=0)

x_train_all = np.concatenate((x_train_ar,x_train_en),axis=0)
y_train_all = np.concatenate((y_train_ar,y_train_en),axis=0)

print(x_train_ar.shape , x_test_ar.shape)
print(y_train_ar.shape , y_test_ar.shape)

print(x_train_en.shape , x_test_en.shape)
print(y_train_en.shape , y_test_en.shape)
      
print(x_train_all.shape , x_test_all.shape)
print(y_train_all.shape , y_test_all.shape)

(1692, 138) (386, 138)
(1692,) (386,)
(1692, 138) (386, 138)
(1692,) (386,)
(3384, 138) (772, 138)
(3384,) (772,)


In [7]:
#Case마다 여기서 변경
train_x = x_train_all
test_x = x_test_all

train_y = y_train_all
test_y = y_test_all

cs = StandardScaler()
train_x = cs.fit_transform(train_x)
test_x = cs.transform(test_x)



#모두 False나와야 함
print(np.isnan(train_x).all(),np.isnan(test_x).all(),
      np.isnan(train_y).all(),np.isnan(test_y).all())
#모두 True 나와야 함
print(np.isfinite(train_x).all(),np.isfinite(test_x).all(),
      np.isfinite(train_y).all(),np.isfinite(test_y).all())

print(f'train data length: {len(train_x)} , {len(train_y)}')
print(f'test data length: {len(test_x)} , {len(test_y)}')

False False False False
True True True True
train data length: 3384 , 3384
test data length: 772 , 772


SVM

In [8]:
pd.set_option('display.max.colwidth', 100)

In [9]:
param_grid = {'kernel':['rbf'], 'C':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
              'gamma':['auto','scale',0.0001,0.0005,0.001,0.002,0,0.005,0.007,0.01,0.02,0.05]}

svm_clf = SVC(random_state=100)
grid = GridSearchCV(svm_clf, param_grid=param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
grid.fit(train_x, train_y)

print(grid.best_score_,grid.best_params_)
print(pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score')[['rank_test_score','params','mean_test_score']][:5])

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   47.3s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  3.6min finished


0.7361178362598655 {'C': 3, 'gamma': 0.002, 'kernel': 'rbf'}
     rank_test_score                                      params  \
29                 1   {'C': 3, 'gamma': 0.002, 'kernel': 'rbf'}   
89                 2   {'C': 8, 'gamma': 0.002, 'kernel': 'rbf'}   
124                3  {'C': 11, 'gamma': 0.001, 'kernel': 'rbf'}   
136                4  {'C': 12, 'gamma': 0.001, 'kernel': 'rbf'}   
172                5  {'C': 15, 'gamma': 0.001, 'kernel': 'rbf'}   

     mean_test_score  
29          0.736118  
89          0.735823  
124         0.735527  
136         0.735232  
172         0.734937  


In [10]:
svm_pred = grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, svm_pred)*100, '%')

Accuracy: 70.85492227979275 %


rf

In [ ]:
rf_param_grid = {
    'n_estimators' : [280,290,300],
    'max_depth' : [22,23],      #default : " None",
    'min_samples_leaf' : [1,2],     #default : 1
    'min_samples_split' : [2,3,4]  #default : 2
}

rf_clf = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(rf_clf, param_grid=rf_param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
rf_grid.fit(train_x, train_y)

print(rf_grid.best_score_,rf_grid.best_params_)
rf_df = pd.DataFrame(rf_grid.cv_results_).sort_values(by='rank_test_score')
print(rf_df[['params','mean_test_score']][:5])

In [ ]:
rf_pred = rf_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, rf_pred)*100, '%')

XGB

In [ ]:
xgb_param={
    'n_estimators' : [310,320,330],
    'learning_rate' : [0.1,0.15],
    'max_depth' : [7,8,9],
    #'gamma' : [0],
    'colsample_bytree' : [0.8,0.9],
    
}
xgb_clf= xgb.XGBClassifier(random_state=42)#tree_method='gpu_hist', gpu_id=0,
xgb_grid = GridSearchCV(xgb_clf, param_grid=xgb_param,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
xgb_grid.fit(train_x, train_y)

print(xgb_grid.best_score_,xgb_grid.best_params_)
xgb_df = pd.DataFrame(xgb_grid.cv_results_).sort_values(by='rank_test_score')
print(xgb_df[['params','mean_test_score']][:5])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   53.7s


In [ ]:
xgb_pred = xgb_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, xgb_pred)*100, '%')

Logistic regression

In [ ]:
param_grid = {'penalty':['l1','l2'], 
              'C':[0.001,0.005,0.01,0.05,0.1,0.3,0.5], 
              'max_iter':[50,75,100,150,200,250,300,500]}

log_clf = LogisticRegression(random_state = 42)
log_grid = GridSearchCV(log_clf, param_grid=param_grid,scoring = 'accuracy',verbose = 1,n_jobs =-1) 
log_grid.fit(train_x, train_y)
print(log_grid.best_score_,log_grid.best_params_)
print(pd.DataFrame(log_grid.cv_results_).sort_values(by='rank_test_score')[['rank_test_score','params','mean_test_score']][:5])

In [ ]:
log_pred = log_grid.best_estimator_.predict(test_x)
print('Accuracy:', accuracy_score(test_y, log_pred)*100, '%')